In [3]:
%matplotlib inline
import os
import re
import gc
import glob
import math
import pickle
import numpy  as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from collections import defaultdict

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [4]:
from matplotlib.backends.backend_pdf import PdfPages

In [5]:
tf.__version__

'2.8.0'

In [6]:
tf.compat.v1.disable_eager_execution()
def data_extractor_fn(filelist,prefetch=1, read_threads=4, distribute=(1, 0), nlimit = 100):
    def parser(ser):
        """
        Decode & Pass datast in tf.record
        *Cuation*
        floating point: tfrecord data ==> tf.float64
        """
        features = {
            "shape": tf.io.FixedLenFeature([3], tf.int64),
            "patch": tf.io.FixedLenFeature([], tf.string),
            "filename": tf.io.FixedLenFeature([], tf.string),
            "coordinate": tf.io.FixedLenFeature([2], tf.int64),
        }
        decoded = tf.io.parse_single_example(ser, features)
        patch = tf.reshape(
            tf.io.decode_raw(decoded["patch"], tf.float64), decoded["shape"]
        )
        # keep the value as float64

        # get other configs
        filename = decoded["filename"]
        coordinate = decoded["coordinate"]
        return patch, filename, coordinate
    
    dataset = (
        tf.data.Dataset.list_files(filelist, shuffle=True)
            .shard(*distribute)
            .apply(
            tf.data.experimental.parallel_interleave(
                lambda f: tf.data.TFRecordDataset(f).map(parser),
                cycle_length=read_threads,
                sloppy=True,
            )
        )
    )
    # iterator = dataset.make_one_shot_iterator()
    iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
    next_element = iterator.get_next()
    
    # Process
    idx = 0
    patches = None
    with tf.compat.v1.Session() as sess:
        try:
            while True:
                patch, filename, coord  = sess.run(next_element)
                #yield  patch, filename.decode("utf-8"), coord
                _patch = np.expand_dims(patch, axis=0)
                if idx == 0:
                    patches = _patch
                else:
                    patches = np.concatenate([patches, _patch], axis=0)
                idx+=1
                
                if idx >= nlimit:
                    break
        except tf.errors.OutOfRangeError:
            #if crank == 0:
            print(" ###  TF-DEOCDED END--> next process ###", flush=True)
            pass
    return patches

In [7]:
tf_basedir="/tf/cloud-type/prep/pipeline"
tf_datadir=os.path.join(tf_basedir, 'out') # validation data
filelist  = glob.glob(os.path.join(tf_datadir, '*.tfrecord'))

In [8]:
filelist

['/tf/cloud-type/prep/pipeline/out/sample_0-3.tfrecord',
 '/tf/cloud-type/prep/pipeline/out/sample_0-4.tfrecord']

In [9]:
patches = data_extractor_fn(filelist)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.deterministic`.


In [10]:
patches.shape

(100, 128, 128, 3)

In [11]:
def load_latest_model(model_dir, mtype):
    #TODO add restart model dir and restart argument?
    latest = 0, None
    # get trained wegiht 
    for m in os.listdir(model_dir):
        if ".h5" in m and mtype in m:
            epoch = int(m.split("-")[1].replace(".h5", ""))
            latest = max(latest, (epoch, m))

    epoch, model_file = latest

    if not os.listdir(model_dir):
        raise NameError("no directory. check model path again")

    print(" Load {} at {} epoch".format(mtype, epoch))
    model_def = model_dir+'/'+mtype+'.json'
    model_weight = model_dir+'/'+mtype+'-'+str(epoch)+'.h5'
    with open(model_def, "r") as f:
        model = tf.keras.models.model_from_json(f.read())
    model.load_weights(model_weight)
    return model

In [14]:
model_datadir = '/tf/cloud-type/prep/pipeline'
expname =  10057054 # 10056527
model_dir = os.path.join(model_datadir,str(expname) )
encoder = load_latest_model(model_dir, mtype='encoder')
decoder = load_latest_model(model_dir, mtype='decoder')

 Load encoder at 200 epoch
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.8/dist-packages/keras/layers/core/lambda_layer.py:303: UserWarning: models_update is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,


 Load decoder at 200 epoch


In [21]:
model = decoder

### processing ###
layer_names = [layer.name for layer in model.layers]
encs = encoder.predict(patches)

hello = tf.keras.Model(inputs=model.input, outputs=model.get_layer('conv2d_18').output)
decoder_result = hello.predict(encs)

In [22]:
decoder_result.shape

(100, 8, 8, 128)

In [23]:
patches.shape

(100, 128, 128, 3)